In [1]:
import shutup
shutup.please()

import numpy as np
import pandas as pd

import pytrends
from pytrends.request import TrendReq

In [2]:
from src.utils.data import Downloader, FeatureEngeneer
from src.constants import DOW_30_2019, ALLOWED_NEWS_TICKERS

In [3]:
# TICKERS_FOR_COMPARISON = DOW_30_2019
# TICKER_FOR_ANALYSIS = 'aapl'
# # Download datasets
# downloader = Downloader(tickers=TICKERS_FOR_COMPARISON)
# #downloader = Downloader(tickers=['aapl'])

# df = downloader.fetch_data()


In [4]:
# # Add features to datasets
# feature_engeneer = FeatureEngeneer(df)
# df = feature_engeneer.vix(df)
# df = feature_engeneer.turbulence(df)
# print(len(df))
# df = feature_engeneer.technical_indicators(df)
# print(len(df))
# df = feature_engeneer.sentiment_analysis(df,TICKER_FOR_ANALYSIS,key="title")
# print(len(df))
# df = feature_engeneer.sentiment_analysis(df,TICKER_FOR_ANALYSIS,key="body")
# print(len(df))
# df = feature_engeneer.trends(df,TICKER_FOR_ANALYSIS)
# print(len(df))
# df = feature_engeneer.fear_and_greed(df)

# print(len(df))
# df = feature_engeneer.cleanup(df)
# df

In [5]:

# pytrend = TrendReq()

# kw_list=['tsla','tesla','elon musk']
# trend_keyword_list=['tsla','tesla','elon musk']
# historicaldf = pytrend.get_historical_interest(kw_list, year_start=2017, month_start=1, day_start=1, hour_start=0, year_end=2021, month_end=1, day_end=1, hour_end=0, cat=0, geo='', gprop='', sleep=0)


In [6]:
class Scraper:
    def __init__(self, 
        ticker_for_analysis,    
        tickers_for_comparison=DOW_30_2019,
        trend_keyword_list=[],
        main_keyword=False
    ):
        assert ticker_for_analysis.upper() in ALLOWED_NEWS_TICKERS, 'Cannot use this ticker right now, please select one of the following {}'.format(ALLOWED_NEWS_TICKERS)
        self.ticker_for_analysis = ticker_for_analysis
        self.tickers_for_comparison = tickers_for_comparison
        self.trend_keyword_list = trend_keyword_list
        self.main_keyword=main_keyword

    def initial_data_cleanup(self):
        self.__init_dataframe()
        
    def __init_downloader(self):
        self.downloader = Downloader(tickers=self.tickers_for_comparison)
        #downloader = Downloader(tickers=['aapl'])
        self.df = self.downloader.fetch_data()

    def __init_helpers(self):
        self.feature_engeneer = FeatureEngeneer(self.df)

    def __init_features(self):
        # Add features to datasets
        df = self.df.copy()
        
        df = self.feature_engeneer.vix(df)
        df = self.feature_engeneer.turbulence(df)
        df = self.feature_engeneer.fear_and_greed(df)
        df = self.feature_engeneer.technical_indicators(df)
        
        self.df = df

    def __init_sentiment_analysis(self):
        df = self.df.copy()
        df = self.feature_engeneer.sentiment_analysis(df,self.ticker_for_analysis,key="title")
        df = self.feature_engeneer.sentiment_analysis(df,self.ticker_for_analysis,key="body")
        
        self.df = df 
        
    def __init_trends(self):
        df = self.df.copy()
        df = self.feature_engeneer.trends(
            df,
            self.ticker_for_analysis,
            kw_list=self.trend_keyword_list,
            main_keyword=self.main_keyword)
        
        self.df = df
        
        
    def __init_dataframe(self):
        self.__init_downloader()
        self.__init_helpers()
        self.__init_features()
        self.__init_sentiment_analysis()
        self.__init_trends()
        #self.df = self.feature_engeneer.cleanup(self.df)
        
        

In [7]:
scraper = Scraper(ticker_for_analysis='ibm',trend_keyword_list=['ibm'],main_keyword="ibm")

In [8]:
scraper.initial_data_cleanup()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\filip\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [9]:
scraper.df

,date,open,high,low,close,volume,ticker,day,vix,turbulence,...,close_60_sma,neg_title,neu_title,pos_title,compound_title,neg_body,neu_body,pos_body,compound_body,ibm
0,2018-01-05,155.296371,155.736145,154.015289,155.344162,5434807,ibm,4,0.0922,0.062468,...,155.344162,0.000,0.533,0.467,0.718400,0.011000,0.899000,0.089000,0.807400,0.605833
1,2018-01-10,155.745697,157.103256,155.353729,156.959854,4366109,ibm,2,0.0982,0.039646,...,156.302582,0.158,0.602,0.240,0.271600,0.047000,0.823000,0.130000,0.989200,0.654167
2,2018-01-11,157.552582,157.686417,156.118546,156.978973,3969047,ibm,3,0.0988,0.061024,...,156.437860,0.000,0.929,0.071,0.133967,0.002667,0.924667,0.072667,0.746467,0.678750
3,2018-01-12,156.806885,157.495224,155.860428,155.965576,5263367,ibm,4,0.1016,0.075438,...,156.359146,0.000,1.000,0.000,0.000000,0.012000,0.857000,0.131000,0.997700,0.600000
4,2018-01-16,157.839386,158.021027,156.166351,156.644363,8152733,ibm,1,0.1166,0.152541,...,156.399891,0.000,1.000,0.000,0.000000,0.026000,0.860500,0.113750,0.906575,0.377500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,2019-11-29,127.724663,128.604202,127.179733,128.537292,1909264,ibm,4,0.1262,0.014013,...,132.518483,0.217,0.783,0.000,-0.361200,0.000000,0.894000,0.106000,0.648600,0.471667
241,2019-12-06,126.912048,128.001907,126.864243,127.361374,3595625,ibm,4,0.1362,0.056669,...,131.669375,0.000,1.000,0.000,0.000000,0.075000,0.833000,0.093000,0.747400,0.571250
242,2019-12-10,128.030594,128.910141,127.686424,128.021027,3640498,ibm,1,0.1568,0.014782,...,131.377469,0.000,1.000,0.000,0.000000,0.025000,0.872000,0.103000,0.990500,0.625417
243,2019-12-18,128.632889,129.063095,128.250473,128.499039,3244483,ibm,2,0.1258,0.019755,...,130.629063,0.000,1.000,0.000,0.000000,0.014000,0.924000,0.063000,0.908100,0.626250


In [10]:
from stockstats import StockDataFrame as Sdf
stock = Sdf.retype(scraper.df.copy())
unique_ticker = stock.ticker.unique()

In [11]:
stock['cci_30']

date
2018-01-05           NaN
2018-01-10    -18.078521
2018-01-11    143.776502
2018-01-12    185.696640
2018-01-16    118.660920
                 ...    
2019-11-29   -123.513204
2019-12-06   -111.191630
2019-12-10    -84.670788
2019-12-18    -34.413764
2019-12-19    -19.549521
Name: cci_30, Length: 245, dtype: float64

In [12]:
cols = 'cci' # 
stock['adj close']= stock['close']
try:
    stock.drop(columns=['close_30_sma'],inplace=True)
except:
    pass

try:
    stock.drop(columns=['close_60_sma'],inplace=True)
except:
    pass
try:
    stock.drop(columns=['cci_30'],inplace=True)
except:
    pass
stock[cols]

ZeroDivisionError: float division by zero

In [ ]:
from talib import CCI

In [ ]:
!pip install pandas_ta

In [ ]:
stock['close_60_sma']

In [ ]:
ticker="ibm"
trends_df = pd.read_csv('storage/datasets/trends__'+ticker.lower()+'.csv')

In [ ]:
trends_df[trends_df.select_dtypes(include=['int64','float64'])]

In [ ]:
trends_df.select_dtypes(include=['int64','float64']).columns